In [ ]:
import os
import random
import numpy as np
from glob import glob
from PIL import Image, ImageOps
import matplotlib.pyplot as plt

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

import wandb; wandb.login()
from wandb.keras import WandbCallback

In [ ]:
run = wandb.init()
artifact = run.use_artifact('ivangoncharov/Low Light Enhancement with Zero-DCE/Lol_Dataset:v0', type='dataset')
artifact_dir = artifact.download()
artifact_path = os.path.join(artifact_dir, "lol_dataset.zip")

In [ ]:
!unzip $artifact_path